# Task:

- Exploration und Dimensionsreduktion mit PCA / FFT 
- supervised Vorhersage des Wochentags (1(montag)-7(sonntag))

|Teammitglied|MNr|
|-|-|
|Martin Schauer|7961802|
|Joel Bück|4860895|
|Lukas Stamm|8402366|


In [1]:
# Imports
import os
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
from scipy.fft import fft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.pipeline import Pipeline

## Step 1) Load data

Data-Format

-------------

There are two files for each fold, the data file and the labels file. We have split the 440 time series between train and test folds, but you are of course free to merge them to consider a different cross validation setting.
- The PEMS_train textfile has 263 lines. Each line describes a time-series provided as a matrix. The matrix syntax is that of Matlab, e.g. [ a b ; c d] is the matrix with row vectors [a b] and [c d] in that order. Each matrix describes the different occupancies rates (963 lines, one for each station/detector) sampled every 10 minutes during the day (144 columns).
- The PEMS_trainlabel text describes, for each day of measurements described above, the day of the week on which the data was sampled, namely an integer between 1 (Mon.) and 7 (Sun.).

- PEMS_test and PEMS_testlabels are formatted in the same way, except that there are 173 test instances.

- The permutation that I used to shuffle the dataset is given in the randperm file. If you need to rearrange the data so that it follows the calendar order, you should merge train and test samples and reorder them using the inverse permutation of randperm.

In [2]:
def open_data(data_file,label_file)->tuple[pd.DataFrame,list]:
    with open(os.path.join(os.getcwd(),data_file),"r") as of:
        data = [" ".join(d[1:-2].split(";")).split(" ") for d in of.readlines()] # matlab syntax to pd.dataframe
        df = pd.DataFrame(data=data).astype("float")
    with open(os.path.join(os.getcwd(),label_file),"r") as of:
        labels = " ".join(of.read()[1:-2].split(";")).split(" ")# matlab syntax to pd.dataframe
        labels = [float(l) for l in labels]
    return df, labels

In [3]:
# Load training data and labels from the specified files
train_set, trainlabels = open_data("PEMS_train", "PEMS_trainlabels")

# Load test data and labels from the specified files
test_set, testlabels = open_data("PEMS_test", "PEMS_testlabels")

## Step 2) Dimension reduction

### PCA

In [4]:
# Import PCA (Principal Component Analysis) from sklearn if not already done
# PCA is used to reduce the dimensionality of the dataset while retaining 90% of the original data's variance
pca = PCA(n_components=0.9)  # Keep 90% of the information (variance) in the data

# Apply PCA transformation to the training dataset
# `fit_transform` computes the PCA transformation based on the training set and then applies it to the same data
train_set_pca = pca.fit_transform(train_set)

# Apply the learned PCA transformation to the test dataset
# `transform` applies the same transformation from the training set PCA without re-computing
test_set_pca = pca.transform(test_set)

# Print the shapes of the transformed datasets and the number of labels
# This helps verify the dimensionality reduction and ensures the data and labels align
print(f"train: {train_set_pca.shape}\n"       # Prints the shape of the PCA-transformed training set
      f"test: {test_set_pca.shape}\n"        # Prints the shape of the PCA-transformed test set
      f"labels: ({len(trainlabels)} & {len(testlabels)})")  # Prints the number of labels in train and test sets


train: (267, 98)
test: (173, 98)
lables: (267 & 173)


### FFT

In [10]:

# --> exploration of method in seperate notebook, no satisfactory result achieved

0                       1                       2       \
0    7849.3605-   0.0000j -116.675672- 49.075851j  -54.445511- 26.716478j   
1    7967.2563-   0.0000j -127.593348- 73.672773j  -77.698074- 13.367225j   
2    7693.3056-   0.0000j  -84.171713- 47.764552j  -29.596806- 30.339704j   
3    8177.6446-   0.0000j -151.689298-125.539831j -290.876889+ 44.490078j   
4    5540.0224-   0.0000j   -1.714268- 55.836627j  -30.730693-218.342970j   
..                    ...                     ...                     ...   
262  7608.4676-   0.0000j -201.859791-118.456064j -163.487743- 95.042448j   
263  8005.5442-   0.0000j  -98.531070-169.753031j  -43.225891- 79.028486j   
264  8391.3841-   0.0000j -193.722814-330.553437j -157.355762+ 32.860201j   
265  6189.2182-   0.0000j  -39.295321+ 59.713217j  -54.995262- 40.395855j   
266  8804.8022-   0.0000j -128.122092-100.602936j  -66.000122+ 45.232584j   

                     3                       4                      5       \
0    115.417684- 68.160759j  152.100970- 18.477380j -80.872637+ 91.642832j   
1    142.108033- 99.231013j  180.313009-  6.779441j -99.737953+ 90.340507j   
2     95.402665- 90.046476j  141.280689+ 22.524088j -37.782701- 26.885945j   
3    152.297116- 93.530641j  378.096157-145.912605j -19.091108+ 16.415380j   
4     23.923997-152.879125j  -48.475982- 55.102648j -70.917302+ 36.534380j   
..                      ...                     ...                    ...   
262  156.577998- 13.209539j  200.537384-103.075510j  42.332342- 14.414352j   
263   33.081777- 29.327416j  218.903038- 57.587849j  36.183356+ 26.466569j   
264  134.514413- 91.679670j  170.077894-133.213583j -26.882650+ 46.734370j   
265   59.146708- 52.457728j  113.892097- 13.275144j  47.956691+  1.290990j   
266   39.981468- 72.820203j  275.331486-113.560059j  51.228467+103.551012j   

                     6                       7                       8       \
0     95.455432+ 66.964620j -132.737180+ 33.479892j  -64.453599- 61.901430j   
1     88.004020+ 76.669816j -135.369061+ 26.738596j  -39.234323- 79.332410j   
2    120.992048+ 63.026940j -149.545106+ 29.405590j  -37.854055- 11.027108j   
3     12.454774+258.620485j -103.453731+ 66.180403j  -33.572481-240.167501j   
4     56.599857+198.508685j  -61.621603+118.332303j  104.711238+  6.052865j   
..                      ...                     ...                     ...   
262   27.786675+161.366228j -123.272881+ 23.272678j   66.565233- 69.513658j   
263  111.363551+ 83.411665j -188.868238- 10.906513j    1.334428- 41.287405j   
264  195.045848+129.980967j  -92.023499- 58.380690j  -15.075786- 67.947704j   
265  125.867234-  4.184008j -105.574843+ 25.520124j  -59.088627-  7.748515j   
266  179.609265- 27.278655j -141.106410+119.897837j   55.528593- 67.697242j   

                     9       ...                  138662  \
0   -117.034025- 57.777503j  ... -148.273949+ 44.868727j   
1   -112.292884- 53.669801j  ... -192.871744+ 45.086694j   
2   -118.797205- 63.621331j  ... -124.548367+ 13.779506j   
3   -153.345471-106.742700j  ...  -43.565481-142.125212j   
4     66.756985- 51.835813j  ...    2.645941+136.481777j   
..                      ...  ...                     ...   
262 -176.432965- 27.128810j  ... -134.899338+ 19.413267j   
263 -152.677327- 12.586834j  ...  -50.576112+ 61.266765j   
264   52.853294+ 65.614002j  ...  -86.897811+178.313427j   
265  -83.361324- 25.596824j  ... -114.967525-  3.611389j   
266 -124.623492+ 77.552159j  ... -111.683944+ 68.836264j   

                     138663                  138664                  138665  \
0   -117.034025+ 57.777503j  -64.453599+ 61.901430j -132.737180- 33.479892j   
1   -112.292884+ 53.669801j  -39.234323+ 79.332410j -135.369061- 26.738596j   
2   -118.797205+ 63.621331j  -37.854055+ 11.027108j -149.545106- 29.405590j   
3   -153.345471+106.742700j  -33.572481+240.167501j -103.453731- 66.180403j   
4     66.756985+ 51.835813j  104.711238-  6.052865j  -61.621603-118.332303j   
..           

## Step 3) Model creation

In [5]:
# Define the model and hyperparameter grid for a Random Forest Classifier
model_outline = (
    RandomForestClassifier(
        max_features='sqrt',      # Number of features to consider at each split ('sqrt' is the square root of total features)
        n_estimators=100,         # Number of trees in the forest
        max_depth=None,           # Maximum depth of the tree (None means nodes are expanded until all leaves are pure)
        min_samples_split=2,      # Minimum number of samples required to split an internal node
        min_samples_leaf=1,       # Minimum number of samples required to be at a leaf node
        bootstrap=True,           # Whether bootstrap samples are used when building trees
        random_state=42           # Random seed for reproducibility
    ), 
    {
        # Define a grid of hyperparameters for the classifier to be used in hyperparameter tuning
        'classifier__n_estimators': [100, 200, 300, 400, 500],  # Number of trees in the forest
        'classifier__max_depth': [None, 10, 25, 50],            # Maximum depth of trees
        'classifier__min_samples_split': [2, 5, 10],            # Minimum samples required to split a node
        'classifier__min_samples_leaf': [1, 2, 4],              # Minimum samples required to form a leaf node
        'classifier__max_features': [1, 5, 10, 'sqrt', 'log2'], # Number of features to consider at each split
        'classifier__bootstrap': [True, False]                  # Whether to use bootstrap sampling
    }
)

# Create a scoring function for model evaluation
# 'make_scorer' wraps the accuracy_score metric into a format usable for grid search or cross-validation
scorer = make_scorer(
    accuracy_score,              # The metric used to evaluate the model's performance (accuracy)
    greater_is_better=True       # Indicates that a higher score is better for model performance
)


In [6]:
def fit_model(model_name, model, param_grid, features_train, target_train, scoring, cv=10, n_iter=100, random_state=42, n_jobs=-1):
    
    # Create a pipeline with a single step for the classifier
    pipeline = Pipeline(steps=[
        ('classifier', model)  # The classifier to be tuned
    ])
    
    # Initialize RandomizedSearchCV to perform hyperparameter tuning
    # - Uses the pipeline as the estimator
    # - Searches the parameter grid for optimal hyperparameters
    randomized_search = RandomizedSearchCV(
        estimator=pipeline,          # The pipeline (containing the model)
        param_distributions=param_grid,  # Hyperparameter grid for tuning
        scoring=scoring,             # Scoring function for evaluation
        cv=cv,                       # Number of cross-validation folds
        n_iter=n_iter,               # Number of parameter combinations to test
        random_state=random_state,   # Random seed for reproducibility
        n_jobs=n_jobs                # Parallel processing (-1 uses all processors)
    )
    
    # Fit the RandomizedSearchCV to the training data
    randomized_search.fit(features_train, target_train)
    
    # Extract the best score and corresponding parameters
    best_score = randomized_search.best_score_  # Best score achieved
    best_params = randomized_search.best_params_  # Best hyperparameter combination
    
    # Display the model's name, accuracy score, and best parameters
    print(f'{model_name}:\naccuracy : {best_score:.2f}')  # Print the accuracy score
    print('Best Parameters :', best_params)              # Print the best parameters
    print('\n')  # Add spacing for readability
    
    # Return the trained RandomizedSearchCV object and the best parameters
    return randomized_search, best_params

# Train the model using the PCA-transformed training data
model_pca, best_params_pca = fit_model(
    "RandomForestClassifier (PCA)",  # Model name
    model_outline[0],                # The Random Forest model
    model_outline[1],                # The hyperparameter grid
    train_set_pca,                   # PCA-transformed training features
    trainlabels,                     # Training labels
    scoring=scorer                   # Scoring function
)


RandomForestClassifier:
accuracy : 0.88
Best Parameters : {'classifier__n_estimators': 400, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 1, 'classifier__max_features': 'sqrt', 'classifier__max_depth': 50, 'classifier__bootstrap': False}




## Step 4) Evaluation

In [7]:
# Use the trained model (model_pca) to make predictions on the PCA-transformed test dataset
prediction_pca = model_pca.predict(test_set_pca)

# Calculate the accuracy of the predictions by comparing them to the actual test labels
# accuracy_score measures the fraction of correctly classified samples
acc_pca = accuracy_score(testlabels, prediction_pca)

# Print the accuracy score for the PCA-transformed test set
# Format the accuracy to two decimal places for readability
print(f"Accuracy (PCA): {acc_pca:.2f}")


Accuracy (PCA): 0.82
